In [ ]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.chat_models import ChatOpenAI
from pydantic import BaseModel, ValidationError
from typing import Optional
import pandas as pd
import re
import time
import openai
import os

In [6]:

survey_result = pd.read_csv("survey_data.csv")

examples = []

for _, row in survey_result.iterrows():
    example_dict = {
        "Nama": row['Nama'],
        "Usia": row['Usia'],
        "Jenis_Kelamin": row['Jenis Kelamin'],
        "Asal": row['Asal'],
        "Aktif_Lari_Rutin": row['1. Aktif lari rutin?'],
        "Alasan_Mulai_Lari": row['2. Alasan mulai lari?'],
        "Tergabung_Komunitas": row['3. Tergabung komunitas lari?'],
        "Pengaruh_Media_Sosial": row['4. Pengaruh media sosial?'],
        "Hal_Disukai": row['5. Hal disukai dari running cult?'],
        "Share_Di_Medsos": row['6. Seberapa sering share di medsos?'],
        "Lari_Gaya_Hidup_Atau_Olahraga": row['7. Lari sebagai gaya hidup atau olahraga?'],
        "Pernah_FOMO": row['8. Pernah FOMO karena medsos lari?'],
        "Dampak_Trend": row['9. Dampak tren running cult?'],
        "Kemungkinan_Ikut_Race": row['10. Kemungkinan ikut race?']
    }

    formatted_string = ', '.join([f'{k}: {v}' for k, v in example_dict.items()])
    examples.append({"example": formatted_string})


In [7]:
class sample_survey(BaseModel):
    Nama: str
    Usia: int
    Jenis_Kelamin: str
    Asal: str
    Aktif_Lari_Rutin: str
    Alasan_Mulai_Lari: str
    Tergabung_Komunitas: str
    Pengaruh_Media_Sosial: str
    Hal_Disukai: str
    Share_Di_Medsos: str
    Lari_Gaya_Hidup_Atau_Olahraga: str
    Pernah_FOMO: str
    Dampak_Trend: str
    Kemungkinan_Ikut_Race: str

In [ ]:
SYNTHETIC_FEW_SHOT_PREFIX = "Contoh data survei tentang running cult:\n\n"
SYNTHETIC_FEW_SHOT_SUFFIX = """Buatlah satu data survei baru berdasarkan pola yang ada.
Subjek: {subject}
Detail tambahan: {extra}

Pastikan:
- Jawaban beragam dan tidak identik dengan contoh.
- Gunakan variasi usia, asal, dan motivasi lari.
- Jangan MENGULANG NAMA responden atau kombinasi jawaban yang sama.
- Nama sesuai dengan JENIS KELAMIN responden.

Output hanya berisi SATU entri dan tidak lebih.
"""

OPENAI_TEMPLATE = PromptTemplate(
    input_variables=["example"], 
    template="{example}"
    )

prompt_template = FewShotPromptTemplate(
    prefix=SYNTHETIC_FEW_SHOT_PREFIX,
    examples=examples,
    suffix=SYNTHETIC_FEW_SHOT_SUFFIX,
    input_variables=["subject", "extra"],
    example_prompt=OPENAI_TEMPLATE,
)

subject = "Generasi muda yang aktif di media sosial dan suka berlari"
extra = "Mereka sering membagikan pengalaman lari mereka di media sosial dan terpengaruh oleh tren running cult."

In [ ]:
llm = ChatOpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=os.getenv("QWEN_API_KEY"),
)

/var/folders/b9/5pxngp4x62q2qbmwnt1x80vr0000gn/T/ipykernel_95315/1665353076.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [26]:
synthetic_data = []
for i in range(50):
    prompt = prompt_template.format(subject=subject, extra=extra)
    response = llm.predict(prompt)

    print(f"\n[iterazione {i+1}] Output di dati sintetici:\n{response}")

    fields = [field.strip() for field in response.strip().split(",")]
    entry_dict = {}
    for field in fields:
        if ": " in field:
            key, value = field.split(": ", 1)
            entry_dict[key.strip()] = value.strip()

    try:
        validated = sample_survey(**entry_dict)
        synthetic_data.append(validated.model_dump())
    except ValidationError as e:
        print(f"[ERROR] Validation error on entry {i+1}:\n{e}")
    
    time.sleep(1.5)


[iterazione 1] Output di dati sintetici:
Nama: Dini, Usia: 22, Jenis_Kelamin: Perempuan, Asal: Semarang, Jawa Tengah, Aktif_Lari_Rutin: Ya, Alasan_Mulai_Lari: Tren media sosial, Tergabung_Komunitas: Ya, aktif, Pengaruh_Media_Sosial: Sangat besar, Hal_Disukai: Dukungan komunitas, Event-event lari, Share_Di_Medsos: Setiap kali lari, Lari_Gaya_Hidup_Atau_Olahraga: Lebih ke gaya hidup, Pernah_FOMO: Sering, Dampak_Trend: Positif, Kemungkinan_Ikut_Race: Pasti ikut

[iterazione 2] Output di dati sintetici:
Nama: Dini, Usia: 20, Jenis_Kelamin: Perempuan, Asal: Malang, Jawa Timur, Aktif_Lari_Rutin: Ya, Alasan_Mulai_Lari: Ingin menurunkan berat badan dan terinspirasi dari influencer, Tergabung_Komunitas: Ya, aktif, Pengaruh_Media_Sosial: Sangat besar, Hal_Disukai: Penghargaan dan pujian dari followers, Event-event lari yang trendi, Share_Di_Medsos: Rutin setiap lari, Lari_Gaya_Hidup_Atau_Olahraga: Lebih ke olahraga, Pernah_FOMO: Sering, Dampak_Trend: Positif, Kemungkinan_Ikut_Race: Pasti ikut



In [23]:
df = pd.DataFrame(synthetic_data)

In [24]:
df

,Nama,Usia,Jenis_Kelamin,Asal,Aktif_Lari_Rutin,Alasan_Mulai_Lari,Tergabung_Komunitas,Pengaruh_Media_Sosial,Hal_Disukai,Share_Di_Medsos,Lari_Gaya_Hidup_Atau_Olahraga,Pernah_FOMO,Dampak_Trend,Kemungkinan_Ikut_Race
0,Siti,18,Perempuan,Palembang,Ya,Menjaga kesehatan tubuh,Tidak,Sangat besar,Outfit lari,Setiap kali lari,Lebih ke olahraga,Sering,Positif,Pasti ikut
1,Dinda,24,Perempuan,Makassar,Ya,Ingin hidup sehat dan mencari teman baru,Ya,Sangat besar,Dukungan komunitas,Setiap kali lari,Lebih ke gaya hidup,Sering,Positif,Pasti ikut
2,Anisa,26,Perempuan,Bandar Lampung,Ya,Ingin mencari teman baru,Ya,Sangat besar,Medali lari,Setiap kali lari,Keduanya seimbang,Sering,Positif,Mungkin ikut
3,Dini,24,Perempuan,Palembang,Ya,Ingin hidup sehat dan menambah teman,Ya,Sangat besar,Gaya hidup sehat,Setiap kali lari,Lebih ke gaya hidup,Sering,Positif,Pasti ikut
4,Dini,24,Laki-laki,Makassar,Ya,Ingin mencapai pencapaian pribadi,Ya,Sangat besar,Medali finisher,Setiap kali lari,Lebih ke olahraga,Sering,Positif,Pasti ikut
5,Dinda,26,Laki-laki,Palembang,Ya,Untuk meningkatkan stamina dan terinspirasi da...,Tidak,Cukup besar,Outfit stylish,Setiap kali lari,Lebih ke olahraga,Kadang-kadang,Positif,Mungkin ikut
6,Dini,26,Perempuan,Solo,Ya,Ingin menambah teman dan menjaga kesehatan,Ya,Sangat besar,Outfit lari,Setiap kali lari,Lebih ke olahraga,Sering,Positif,Pasti ikut
7,Maya,24,Perempuan,Surakarta,Ya,Ingin hidup sehat dan mencari teman dengan min...,Ya,Sangat besar,Gaya hidup stylish,Setiap kali lari,Lebih ke gaya hidup,Sering,Campur (positif & negatif),Pasti ikut
8,Dini,24,Perempuan,Palembang,Ya,Tren media sosial,Ya,Sangat besar,Outfit stylish,Setiap kali lari,Lebih ke gaya hidup,Sering,Positif,Pasti ikut
9,Dita,24,Laki-laki,Surakarta,Ya,Ingin hidup sehat,Ya,Cukup besar,Dukungan komunitas,Kadang-kadang,Lebih ke olahraga,Jarang,Positif,Pasti ikut
